Заполните данные о необходимых переменных:

In [11]:
Today = '2024-03-31'
Mounth = 'Apr'

print_Debt = True
print_counterclaims = False

In [12]:
import pandas as pd
import numpy as np
from datetime import date

In [13]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

Открываю следующие файлы:

In [14]:
query = f"""SELECT * FROM "RISKACCESS"."sapPositionArrears" WHERE "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') AND "productType" in (130,131,132,133)"""
data_export = export_from_RISKCUSTOM(query)
data_export['index_SQL'] = data_export.index

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [15]:
data_Debt = data_export.loc[data_export['productTypeName'].isin(['Транш по займам', 'Транш по кредитам'])].reset_index(drop=True)

data_Debt = data_Debt.rename(columns={"businessPartnerName": "Counterparty", "companyName": "Entity", "positionCurrency": "Currency", "purchaseValuePositionCurrency": "amoutn_outstanding"}).reset_index(drop=True)
data_Debt.amoutn_outstanding = data_Debt.amoutn_outstanding.abs()

data_Debt['termEnd'] = data_Debt['termEnd'].fillna('_')
data_Debt = data_Debt.pivot_table(index=['Entity', 'relationshipPartner', 'Counterparty', 'accountAssignmentLinkName', 'Currency', 'termEnd'], values=['purchaseValueDisplayCurrency', 'amoutn_outstanding', 'index_SQL'], aggfunc={'purchaseValueDisplayCurrency': sum, 'amoutn_outstanding': sum, 'index_SQL':list}).reset_index()

data_Debt = add_in_currency_column(data_Debt, col_with_CCY='Currency', col_with_VAL='amoutn_outstanding', DATE=Today, CCY_to='USD', date_is_column=False)
data_Debt['amoutn_outstanding_in_USD'] = data_Debt.amoutn_outstanding_in_USD * np.sign(data_Debt.purchaseValueDisplayCurrency)

data_Debt['Entity_code'] = merge_Mapping(data_Debt, col='Entity')
data_Debt['Entity_group'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='ocpSegment')
data_Debt['Counterparty_code'] = merge_Mapping(data_Debt, col='Counterparty')
data_Debt['Counterparty_Group'] = merge_SalesUnits(data_Debt, col='Counterparty_code', merge_col='ocpSegment')

data_Debt = Period(data_Debt, day_for_count=Today, col_with_date='termEnd')

data_Debt['Holding'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='holding')
data_Debt['termEnd'] = data_Debt['termEnd'].fillna('_')

new_columns = ['dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']
data_Debt = data_Debt.reindex(columns=(data_Debt.columns.tolist() + new_columns))
data_Debt['Source'] = 'SAP'

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14188\580765983.py:7: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_Debt = data_Debt.pivot_table(index=['Entity', 'relationshipPartner', 'Counterparty', 'accountAssignmentLinkName', 'Currency', 'termEnd'], values=['purchaseValueDisplayCurrency', 'amoutn_outstanding', 'index_SQL'], aggfunc={'purchaseValueDisplayCurrency': sum, 'amoutn_outstanding': sum, 'index_SQL':list}).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14188\580765983.py:7: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_Debt = data_Debt.pivot_table(index=['Entity', 'relationshi

In [16]:

manual1 = data_Debt.loc[data_Debt.Entity_code == 'External' ,['Entity', 'Entity_code']].drop_duplicates()
manual2 = data_Debt.loc[data_Debt.Counterparty_code == 'External' ,['Counterparty', 'Counterparty_Group']].drop_duplicates()
# pd.concat([manual1, manual2])

Записываю данные в output file:

In [17]:
Sheet_in_output_file = 'Debt'

if print_Debt == True:
    for group in ['SUEK', 'EUROCHEM']:
        data_Debt_group = data_Debt[data_Debt.Holding == group]
        data_Debt_print = data_Debt_group[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amoutn_outstanding_in_USD', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
        Output_file = "_".join([str(date.today()), group, 'SAP_Debt', f'{Mounth}.xlsx'])
        data_Debt_print.to_excel(Output_file, Sheet_in_output_file, index=False)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14188\2318075429.py:8: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  data_Debt_print.to_excel(Output_file, Sheet_in_output_file, index=False)


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14188\2318075429.py:8: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  data_Debt_print.to_excel(Output_file, Sheet_in_output_file, index=False)


In [18]:
assert 2 == 1

AssertionError: 

Проверка на наличие тех же записей в Quantum:

In [ ]:
quantum_data_export = pd.read_excel('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Quantum\\2024-03-21_EuroChem_quantum_Debt_Feb.xlsx')
quantum_data_export.head(2)

,entity,Entity_group,counterparty,Counterparty_Group,amoutn outstanding,Currency,amount USD eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,BRHRG,SAMSALES,Morgan Stanley BR,External,-8011.33,BRL,-1616.101732,2031-12-31,2862,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum
1,BRHRG,SAMSALES,Morgan Stanley BR,External,-9958.01,BRL,-2008.799688,2032-12-31,3228,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum


In [ ]:
quantum_data_export['E_Cgroup'] = quantum_data_export.entity + quantum_data_export.Counterparty_Group + quantum_data_export.Currency + quantum_data_export.termEnd.astype(str) + quantum_data_export['amoutn outstanding'].abs().astype(str)
data_Debt['E_Cgroup'] = data_Debt.Entity_code + data_Debt.Counterparty_Group + data_Debt.Currency + data_Debt.termEnd.astype(str) + data_Debt.amoutn_outstanding.abs().astype(str)

iner_list = list(set(data_Debt['E_Cgroup']).intersection(set(quantum_data_export.E_Cgroup)))

SAP_iner = data_Debt[data_Debt.E_Cgroup.isin(iner_list)]
Quantum_iner = quantum_data_export[quantum_data_export.E_Cgroup.isin(iner_list)]

In [ ]:
iner_list_2 = SAP_iner.index.tolist()

SAP_iner = data_Debt[data_Debt.index.isin(iner_list_2)]
data_Debt_not_iner = data_Debt[~data_Debt.index.isin(iner_list_2)]

SAP_iner.shape, Quantum_iner.shape

((16, 23), (16, 17))

In [ ]:
SAP_iner = SAP_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
data_Debt_not_iner = data_Debt_not_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]


In [ ]:
data_Debt_not_iner.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)
new_list(SAP_iner, Output_file, 'Inersection')
new_list(quantum_data_export, Output_file, 'Debt_Quantum')

Проверка встречных требований:

In [ ]:
data_Debt.Holding.unique()

array(['SUEK', 'EUROCHEM', 'External'], dtype=object)

In [ ]:
# Фильтр на холдинг
for group in ['SUEK', 'EUROCHEM']:
    index_dataDebt_to_print = []
    data_E_to_C = data_Debt[data_Debt.Holding == group]
    # Фильт по Counterparty_Group - только внутригрупповые
    data_E_to_C = data_E_to_C.query('Counterparty_Group != "External"')
    data_E_to_C['Index_copy'] = data_E_to_C.index

    # Кокатенация значений
    data_E_to_C['concat_col_EC'] = data_E_to_C[['Entity_code', 'Counterparty_code', 'Currency',  'amoutn_outstanding', 'termEnd']].astype(str).agg('.'.join, axis=1)
    data_E_to_C['concat_col_CE'] = data_E_to_C[['Counterparty_code', 'Entity_code', 'Currency',  'amoutn_outstanding', 'termEnd']].astype(str).agg('.'.join, axis=1)

    # Данные с повторяющимися значениями
    doubles_data = data_E_to_C[data_E_to_C.concat_col_EC.isin(data_E_to_C.concat_col_CE)]
    doubles_data = doubles_data.sort_values(by=['Currency',  'amoutn_outstanding', 'termEnd'])
    # Проверка на отсутсвие необходимости суммировать значения amoutn_outstanding
    group_data = data_E_to_C.pivot_table(index=['Entity_code', 'Counterparty_code','accountAssignmentLinkName', 'Currency', 'termEnd'], values='Index_copy', aggfunc={'Index_copy': list}).reset_index()
    group_data['Index_count'] = group_data['Index_copy'].apply(len)
    assert group_data.Index_count.unique().tolist() == [1]
    # Проверка наличия пары у кадого значения
    group_data = doubles_data.pivot_table(index=['Currency', 'amoutn_outstanding', 'termEnd'], values='Index_copy', aggfunc={'Index_copy': list}).reset_index()
    group_data['Index_count'] = group_data['Index_copy'].apply(len)
    assert group_data.Index_count.unique().tolist() == [2]

    # В список index_dataDebt_to_print добавлена только одна запись из равной пары
    group_data['index_to_print'] = group_data['Index_copy'].str[0]
    index_to_print_list = group_data.index_to_print.tolist()
    index_dataDebt_to_print += index_to_print_list
    # Проверка, что взята ровно половина повторяющихся данных
    assert len(index_dataDebt_to_print) == len(doubles_data)/2

    # Не совпадать значения могу из-за: 1. ошибок в Mapping, 2. неравных значений amount, 3. если у них нет пары
    # 1. Поиск оишбок в Mapping:

    # Данные с НЕповторяющимися значениями
    not_doubles_data = data_E_to_C[~data_E_to_C.concat_col_EC.isin(data_E_to_C.concat_col_CE)]
    not_doubles_data = not_doubles_data.sort_values(by=['Currency',  'amoutn_outstanding', 'termEnd'])

    group_data = not_doubles_data.groupby(['Currency', 'amoutn_outstanding', 'termEnd'], as_index=False).agg({'Index_copy': list})
    group_data['Index_count'] = group_data['Index_copy'].apply(len)

    if len(group_data.loc[group_data.Index_count>1, 'Index_copy'].values) != 0:
        in_doubles = list(np.concatenate(group_data.loc[group_data.Index_count>1, 'Index_copy'].values, axis = 0))
        not_doubles_data_manual = not_doubles_data[not_doubles_data['Index_copy'].isin(in_doubles)]
        not_doubles_data_manual_print = not_doubles_data_manual[['Entity', 'Entity_code', 'Counterparty', 'Counterparty_code']]
        print(not_doubles_data_manual)
        # new_list(not_doubles_data_manual_print, '2024-03-27_SAP_Debt_Feb.xlsx', sheet_name='Not_douples_manual') # название файла
    else:
        pass

    # 2. Поиск неравных значений amount для равных пар:

    not_doubles_data['concat_col2_EC'] = not_doubles_data[['Entity_code', 'Counterparty_code', 'Currency','termEnd']].astype(str).agg('.'.join, axis=1)
    not_doubles_data['concat_col2_CE'] = not_doubles_data[['Counterparty_code', 'Entity_code', 'Currency','termEnd']].astype(str).agg('.'.join, axis=1)

    # Данные с повторяющимися значениями
    doubles_data2 = not_doubles_data[not_doubles_data.concat_col2_EC.isin(not_doubles_data.concat_col2_CE)]
    doubles_data2 = doubles_data2.sort_values(by=['Currency', 'termEnd'])
    group_data = doubles_data2.groupby(['Currency', 'termEnd'], as_index=False).agg({'amoutn_outstanding': 'mean', 'Index_copy': list}).sort_values(['Currency', 'amoutn_outstanding', 'termEnd'])
    group_data['Index_count'] = group_data['Index_copy'].apply(len)
    print(group_data)
    # в выборке присутсвуют только двухстрочне значения
    # assert group_data.Index_count.unique().tolist() == [2]

    ### ЗДЕСЬ МОЖНО ВЫБРАТЬ СРЕДНЕЕ 

    # В список index_dataDebt_to_print добавлена только одна запись из равной пары
    group_data['index_to_print'] = group_data['Index_copy'].str[-1]
    index_to_print_list = group_data.index_to_print.tolist()
    index_dataDebt_to_print += index_to_print_list

    # Данные с НЕповторяющимися значениями
    data = not_doubles_data[~not_doubles_data.concat_col2_EC.isin(not_doubles_data.concat_col2_CE)]
    data = data.sort_values(['Currency',  'amoutn_outstanding', 'termEnd'])
    index_to_print_list = data.Index_copy.tolist()
    index_dataDebt_to_print += index_to_print_list

    assert sum(group_data.Index_count.tolist()) == not_doubles_data.shape[0] - len(data)

    # Запись в файл:

    assert len(index_dataDebt_to_print) < len(data_Debt)
    index_dataDebt_to_print = list(set(index_dataDebt_to_print))

    file = "_".join([str(date.today()), group ,'SAP_Debt_counterclaims.xlsx'])

    data_Debt_after_cleaning = data_Debt.loc[index_dataDebt_to_print, :]
    # Возвращение знака amoutn_outstanding
    data_Debt_after_cleaning.amoutn_outstanding = data_Debt_after_cleaning.amoutn_outstanding * np.sign(data_Debt_after_cleaning.purchaseValueDisplayCurrency)
    # Слеивание с данными внешних требований
    data_Debt_after_cleaning = pd.concat([data_Debt_after_cleaning, data_Debt.query('Counterparty_Group == "External"')])

    if print_counterclaims == True:
        not_doubles_data.to_excel(file, 'Not_douples')
        new_list(doubles_data, file,'Douples', index=True)
        # new_list(data_Debt_after_cleaning, file, 'Debt', index=True)
        new_list(data_Debt, file, 'Debt_before', index=True)
        new_list(data_export, file, 'SQL', index=True)

  Currency    termEnd  amoutn_outstanding       Index_copy  Index_count
0      RUB 2024-09-19        9.773307e+08  [214, 211, 438]            3
1      RUB 2028-06-14        1.218530e+10   [391, 389, 74]            3


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16104\3636644211.py:94: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  not_doubles_data.to_excel(file, 'Not_douples')


  Currency    termEnd  amoutn_outstanding  Index_copy  Index_count
1      KZT 2025-01-31         179235200.0  [475, 473]            2
0      KZT 2024-12-31         919300115.4  [472, 474]            2


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16104\3636644211.py:94: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  not_doubles_data.to_excel(file, 'Not_douples')
